In [1]:
import pandas as pd
import os

from formatacao_sumulas import FootballDataProcessor
from algorithm import FootballMatchRating
import numpy as np
import warnings
warnings.filterwarnings("ignore")


id_players = pd.read_excel('id_players.xlsx', index_col=0)


data_folder = '../Data/'
file_names = ['Serie_A']
years = range(2017, 2018)

# processor = FootballDataProcessor('../Data/Serie_A_2014_games.json')
# final_df = processor.process(data_folder, file_names, years)
# final_df.rename(columns={'player_id': 'player'}, inplace=True)


# # Garantir que ambas as colunas 'player' estejam no tipo string
# final_df['player'] = final_df['player'].astype('string')
# id_players['player'] = id_players['player'].astype('string')

# # Realizar o merge sem utilizar índices, mas mantendo a coluna 'player'
# final_df_ = pd.merge(final_df, id_players, on='player', how='left')

# # Definir os nomes dos times que deseja analisar
# team_1_name = 'Criciúma / SC'  # Substitua pelo nome real do time 1 
# team_2_name = 'Palmeiras / SP'  # Substitua pelo nome real do time 2

# # Criar uma instância de FootballMatchRating
# rating_calculator = FootballMatchRating(final_df_, team_1_name, team_2_name)

# # Atualizar as ratings
# updated_match_data = rating_calculator.update_ratings()

# # Garantir que 'player' em updated_match_data está como string
# updated_match_data['player'] = updated_match_data['player'].astype('string')

# # Merge para obter apenas as colunas 'rating' e 'games_played' que precisam ser atualizadas
# updated_values = id_players[['player']].merge(
#     updated_match_data[['player', 'rating', 'games_played']],
#     on='player',
#     how='left'
# )

# # Atualizar apenas as linhas correspondentes em id_players
# id_players.update(updated_values)

In [2]:
df_jogos_2014 = pd.read_json('../Data/Serie_A_2017_games.json')
df_jogos_2014_ = df_jogos_2014.T

for index, row in df_jogos_2014_.iterrows():
    processor = FootballDataProcessor(df_jogos_2014_, index-1)
    final_df = processor.process(data_folder, file_names, years)
    final_df.rename(columns={'player_id': 'player'}, inplace=True)

    final_df['player'] = final_df['player'].astype('string')
    id_players['player'] = id_players['player'].astype('string')

    final_df_ = pd.merge(final_df, id_players, on='player', how='left')

    team_1_name = df_jogos_2014_.iloc[index-1,0]
    team_2_name = df_jogos_2014_.iloc[index-1,1]


    # Criar uma instância de FootballMatchRating
    rating_calculator = FootballMatchRating(final_df_, team_1_name, team_2_name)

    # Atualizar as ratings
    updated_match_data = rating_calculator.update_ratings()
    # Garantir que 'player' em updated_match_data está como string
    updated_match_data['player'] = updated_match_data['player'].astype('string')
    # Merge para obter apenas as colunas 'rating' e 'games_played' que precisam ser atualizadas
    updated_match_data.rename(columns={'player_name_x': 'player_name'}, inplace=True)
    updated_match_data.rename(columns={'time_jogador_x': 'time_jogador'}, inplace=True)

    updated_values = id_players[['player']].merge(
        updated_match_data[['player', 'rating', 'games_played', 'player_name', 'time_jogador']],
        on='player',
        how='left'
    )

    # Atualizar apenas as linhas corespondentes em id_players
    id_players.update(updated_values)

id_players


,player,rating,age,games_played,player_name,time_jogador
0,135687,1464.423672,25,62,9Luis Fabiano Luis Fabiano Clemente,Vasco da Gama / RJ
1,380762,1490.109888,25,51,21Joao Pedro Joao Pedro Heinen Silva,Athletico Paranaense / PR
2,172028,1428.127380,25,68,1Felipe Felipe Garcia dos Pr...,Atlético / GO
3,439179,1490.667663,25,54,20Fael Rafaelson Bezerra Fe...,Vitória / BA
4,178827,1472.573774,25,82,2Lucas Lucas Rios Marques,Fluminense / RJ
...,...,...,...,...,...,...
717,364085,1487.572939,25,58,70Yan Sasse Yan Medeiros Sasse,Coritiba / PR
718,137990,1499.692001,25,66,12Diego Cava ... Diego Cavalieri,Fluminense / RJ
719,165065,1538.074762,25,79,21Pará Marcos Rogerio Ricci...,Flamengo / RJ
720,178620,1461.787775,25,82,29Patric Patric Cabral Lalau,Vitória / BA


In [3]:
x = id_players.sort_values(by='rating')
x.to_excel('teste.xlsx')
x

,player,rating,age,games_played,player_name,time_jogador
290,184174,1412.666246,25,69,3Roger Roger de Carvalho,Atlético / GO
523,339594,1422.519371,25,79,6Edson Edson Carlos Santos ...,Avaí / SC
40,145923,1424.552357,25,72,55Juan Juan Maldonado Jaime...,Avaí / SC
74,502791,1424.882298,25,69,17Saraiva Felipe Saraiva de So...,Ponte Preta / SP
2,172028,1428.127380,25,68,1Felipe Felipe Garcia dos Pr...,Atlético / GO
...,...,...,...,...,...,...
370,295418,1583.941210,25,76,17Ramiro Ramiro Moschen Benetti,Grêmio / RS
252,339242,1586.872348,25,86,8Maycon Maycon de Andrade Ba ...,Corinthians / SP
438,168351,1590.354054,25,73,1M. Grohe Marcelo Grohe,Grêmio / RS
174,507462,1594.558516,25,80,11Angel Romero Angel Rodrigo Romero...,Corinthians / SP
